In [1]:
%gui qt
%matplotlib inline

import ipyvuetify as v

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import sys
from io import StringIO
import numpy as np
from pathlib import Path
import os
import scipy
import base64
import qgrid
import time

import cufflinks as cf
import plotly.tools as tls
from plotly.offline import download_plotlyjs, plot,iplot
cf.go_offline()

import openalea.strawberry #import openalea.strawberry module
from openalea.strawberry.import_mtgfile import import_mtgfile, plant_number_by_varieties
from openalea.strawberry.analysis import (extract_at_module_scale, extract_at_node_scale, extract_at_plant_scale,
                                          df2waffle, plot_waffle, plot_pie,
                                          occurence_module_order_along_time, pointwisemean_plot, crowntype_distribution)
from openalea.strawberry import visu3d, visu2d 
# from openalea.strawberry import analysis 

from oawidgets.mtg import plot
from oawidgets.plantgl import PlantGL

import openalea.plantgl.all as pgl
from openalea.plantgl.all import *

from openalea.mtg.io import read_mtg_file, write_mtg
from openalea.mtg.algo import split, orders, union
from openalea.mtg import MTG

from pgljupyter import SceneWidget


from fpdf import FPDF

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets import HTML
from traitlets import (Any, Bool, Dict, Int, Unicode, List)


In [2]:
class PDF(FPDF):
    title = ""
    
    def set_title(self, title):
        self.title = title
        
    def header(self):
        # Arial bold 15
        self.set_font('Arial', '', 15)
        # Calculate width of title and position
        w = self.get_string_width(self.title) + 6
        self.set_x((210 - w) / 2)
        # Colors of frame, background and text
        self.set_draw_color(255, 255, 255)
        self.set_fill_color(255, 255, 255)
        self.set_text_color(0, 0, 0)
        # Thickness of frame (1 mm)
#         self.set_line_width(1)
        # Title
        self.cell(w, 9, self.title, 1, 1, 'C', 1)
        # Line break
        self.ln(10)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Arial', 'I', 8)
        # Text color in gray
        self.set_text_color(128)
        # Page number
#         self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def add_btext(self, content):
        self.set_font('Arial', 'B', 14)
        self.set_fill_color(255, 255, 255)
        self.set_text_color(0, 0, 0)
        self.cell(0, 6, content, 0, 1, 'L', 1)
        self.ln(2)


    def add_text(self, content):
        self.set_font('Arial', '', 12)
        self.set_fill_color(255, 255, 255)
        self.set_text_color(0, 0, 0)
        self.cell(0, 6, content, 0, 1, 'L', 1)
        self.ln(2)
        
    def add_image(self, img_path):
        self.image(img_path, w=150)
        self.ln(2)


# MTG dicts

In [3]:
def get_vid_of_genotype(mtg, genotypes):
    if len(genotypes)==1:
        vids=[vid for vid in mtg.vertices(scale=1) if mtg.property('Genotype').get(vid) == genotypes[0]]
    else:
        vids=[]
        for genotype in genotypes:
            tmp=[vid for vid in mtg.vertices(scale=1) if mtg.property('Genotype').get(vid) == genotype]
            vids.extend(tmp)
    return vids

def get_genotypes(mtg):
    return list(set(mtg.property('Genotype').values()))

def get_vid_from_nbplant(all_mtg, genotype, p_nb):
    return get_vid_of_genotype(all_mtg, [genotype])[p_nb-1]

In [4]:
global all_mtg
all_mtg = MTG()

files=[]
# START BY LOADING ALL EXISTING MTG FILES IN /dashboard_files
home = str(Path.home())
data_directory = os.path.join(home, "dashboard_files")
file_paths = {}
for file in os.listdir(data_directory):
    if file.endswith(".mtg"):
        file_paths[file] = os.path.join(data_directory, file)
        
    files.append(file)


# Import MTG

In [5]:
def save_p1():
    files_descriptions = {}
    if p1_wgt_files_selection.v_model:
        selected_files = p1_wgt_files_selection.v_model
        nb_files = len(p1_wgt_files_selection.v_model)
        exp_names = set(all_mtg.property('Experiment_name').values())
        genotype_names = get_genotypes(all_mtg)
        nb_plants={}
        for genotype in genotype_names:
            nb_plants[genotype]= len(get_vid_of_genotype(all_mtg, [genotype]))
        files_descriptions['files']=selected_files
        files_descriptions['nb_files']=nb_files
        files_descriptions['experiment_name']=exp_names
        files_descriptions['genotypes']=genotype_names
        files_descriptions['nb_plants']=nb_plants
    
    pdf = PDF()
    pdf.set_title('Import Files')
    pdf.add_page()
    
    for k in files_descriptions:
        pdf.add_btext(k)
        pdf.add_text(str(files_descriptions[k]))
    pdf.output('p1.pdf', 'F')


In [6]:
# LAYOUT widget

# layout_output_wgt = {'border': '1px solid black', 'margin':'0px', 'padding':'0px'}
layout_output_wgt={}

layout_gofigure={'plot_bgcolor': 'rgba(0,0,0,0)'}

In [7]:
# # ----------------------------------------------------------------
# # PAGE 1: Files Selection
# # ----------------------------------------------------------------


def on_change_upload(widget, event, data):
    # add the file to the mtg repository
    pass

def on_change_get_files(widget, event, data):
    # load mtgs
    global all_mtg
    all_mtg = MTG()
    for file in data:
        mtg = read_mtg_file(file_paths[file])
        all_mtg = union(all_mtg, mtg)
    
    # update table
    if all_mtg:
        df = get_table_mtg(all_mtg)
        df_fixed = fix_inferior_character_for_qgrid(df)
        p1_wgt_tableMTG.df = df_fixed
    else:
        p1_wgt_tableMTG.df= pd.DataFrame()
    
    # update genotype selection
    p1_wgt_genotypes_selection.items=get_genotypes(all_mtg)
    p2_wgt_genotype_selection_3D.items=get_genotypes(all_mtg)
    p2_wgt_genotype_selection_2D.items=get_genotypes(all_mtg)
    p4_wgt_genotypes_selection_t1.items=get_genotypes(all_mtg)
    p4_wgt_genotypes_selection_t2.items=get_genotypes(all_mtg)
    p3_wgt_genotypes_selection_t1.items=get_genotypes(all_mtg)
    p3_wgt_genotypes_selection_t2.items=get_genotypes(all_mtg)
    p4_wgt_genotypes_selection_t4.items=get_genotypes(all_mtg)
    p5_wgt_genotypes_selection_t4.items=get_genotypes(all_mtg)
    
    # update file description
    print_files_description()


p1_wgt_files_upload = v.FileInput(row=True, wrap=True, align_center=True, 
            chips=True, 
            multiple=True,
            counter=True,
            v_model=None,
            label="Import Files",
            truncate_length=22)

p1_wgt_files_selection = v.Select(v_model=None, items=files, 
                        label="Select Files", 
                        multiple=True, chips=True, counter=True)

p1_wgt_files_selection.on_event('change', on_change_get_files)

p1_wgt_export = v.Btn(children=['Export Analyses'])


# # ----------------------------------------------------------------
# # PAGE 1: parameters selection
# # ----------------------------------------------------------------

def on_click_allp(widget, event, data):
    if data:
        p1_wgt_plants_slider.disabled = True
        p1_wgt_plants_nb.disabled = True
    else:
        p1_wgt_plants_slider.disabled = False
        p1_wgt_plants_nb.disabled = False
    
def on_change_genotype(widget, event, data):
    # update plant selection 
    p1_wgt_plants_slider.disabled=False
    p1_wgt_plants_nb.disabled=False
    p1_wgt_cb_allplants.disabled=False    
    p1_wgt_plants_slider.max = len(get_vid_of_genotype(all_mtg, [data]))
    p1_wgt_plants_slider.v_model=1
    
    # update mtg preview
    print_preview(all_mtg, genotype=data, p_nb= p1_wgt_plants_slider.v_model ,width="500px", height="800px")
    
    
def on_change_nbplant(widget, event, data):
    # update mtg preview
    print_preview(all_mtg, genotype=p1_wgt_genotypes_selection.v_model, p_nb= data ,width="500px", height="800px")

def on_change_parameters(widget, event, data):
    # update mtg preview
    if all_mtg:
        print_preview(all_mtg, genotype=p1_wgt_genotypes_selection.v_model, p_nb= p1_wgt_plants_slider.v_model ,width="1000px", height="1000px")


p1_wgt_genotypes_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            counter=True,
            v_model="",
            label="Select Genotypes",
            truncate_length=22)

p1_wgt_parameters = v.Select(items=[{"text":"scale 1",
                                     "value":1},
                                    {"text":"scale 2",
                                     "value":2}], 
                        label="Select Parameters", 
                        multiple=False, chips=True,
                        v_model=2)

p1_wgt_plants_slider = v.Slider(max=100,
             min=1,
           label="#p",
           thumb_label=True,
            v_model="",
            disabled=True)

p1_wgt_plants_nb = v.TextField(type_="number", 
                class_="mt-0 pt-0",
                v_model='',
                disabled=True)

widgets.jslink((p1_wgt_plants_slider,'v_model'), (p1_wgt_plants_nb,'v_model'))

p1_wgt_plants_selection=v.Row(children=[v.Col(cols=12, sm=7, md=7,
                                              children=[p1_wgt_plants_slider]),
                                        v.Col(cols=12, sm=1, md=1,
                                              children=[p1_wgt_plants_nb]) 
                                       ])

p1_wgt_cb_allplants = v.Checkbox(v_model=False,label="Select all plants", disabled=True)

#
p1_wgt_cb_allplants.on_event('change', on_click_allp)
p1_wgt_genotypes_selection.on_event('change', on_change_genotype)
p1_wgt_plants_slider.on_event('change', on_change_nbplant)
p1_wgt_plants_nb.on_event('change', on_change_nbplant)
p1_wgt_parameters.on_event('change', on_change_parameters)


p1_col1 = v.Col(cols=12, sm=3, md=3,
                children=[
                      p1_wgt_files_selection,
                      v.Divider(),
                      p1_wgt_files_upload,
                      v.Divider(),
                      p1_wgt_genotypes_selection,
                      v.Divider(),
                      p1_wgt_plants_selection,
                      v.Divider(),
                      p1_wgt_cb_allplants,
                      v.Divider(),
                      p1_wgt_parameters,
                      v.Divider(),
                      p1_wgt_export
                  ])

# # ----------------------------------------------------------------
# # PAGE 1: MTG Preview
# # ----------------------------------------------------------------

def fix_inferior_character_for_qgrid(df):
    """ Return the dataframe with "<" character transformed as "&lt;"
    """
    return df.applymap(replace_inf_alt)
    
def replace_inf_alt(x):
    if isinstance(x, str):
        return x.replace('<','&lt;')
    else:
        return x

def plot_mtg(mtg, genotype="", p_nb=1, width="600px", height="300px"):
    vid = get_vid_from_nbplant(all_mtg, genotype, p_nb)
    p = plot(mtg.sub_mtg(vid), scale=p1_wgt_parameters.v_model, height=height, width=width)
    # ... set permission at 777
    os.chmod('mtg.html', 0o777)
    return p 

def print_preview(mtg, genotype="", p_nb=1, width="800px", height="600px"):
    with p1_wgt_graphMTG:
        p1_wgt_graphMTG.clear_output()
        display(plot_mtg(mtg, genotype=genotype, p_nb=p_nb, width=width, height=height))


# # ----------------------------------------------------------------
# # PAGE 1: Table MTG
# # ----------------------------------------------------------------

tmp_table_path = "mtg_table.csv"
properties = [('Experiment_name','STRING'), ('Sample_date', 'STRING'),('Genotype', 'STRING'),('Modality', 'STRING'),
              ('Plant_ID', 'INT'),('Stade', 'STRING'),('DBI', 'REAL'),('INFLOLG', 'REAL'),('LFTLG_CENTRAL', 'REAL'),
              ('LFTLG_LEFT', 'REAL'),('FLWRNUMBER', 'REAL'),('FLWRNUMBER_OPEN', 'REAL'),('FLWRNUMBER_ABORTED', 'REAL'),
              ('FLWRNUMBER_CLOSED', 'REAL'),('SAMPLING', 'STRING'),]

def get_table_mtg(mtg):
    current_properties = [t for t in properties if t[0] in mtg.property_names()]
    mtg_string = write_mtg(mtg, current_properties)
    with open(tmp_table_path, "w") as f:
        f.write(mtg_string[mtg_string.find('ENTITY-CODE'):])
        
    df = pd.read_csv(tmp_table_path,
           sep="\t",
           header=0, 
           engine="python")
    df=df.fillna('')
    return df

qgrid.set_grid_option('maxVisibleRows', 10)
p1_wgt_tableMTG = qgrid.show_grid(pd.DataFrame(), show_toolbar=False, 
                                  grid_options={'forceFitColumns': False, 'editable':True, 'defaultColumnWidth':50})

p1_panel_table_mtg = v.Container(
#                               style_='width: 600px; height: 300px',
                              label="The MTG as a table",
                              fluid=True,
                              children=[
                                    p1_wgt_tableMTG    
                            ])

# # ----------------------------------------------------------------
# # PAGE 1: Plot MTG GRAPH
# # ----------------------------------------------------------------

p1_wgt_graphMTG = widgets.Output(layout=layout_output_wgt)

#
# print_preview(mtg_test, width="1000px", height="1000px")

p1_panel_graph_mtg = v.Container(
#                               style_='width: 600px; height: 300px ma: 500',
                              label="The MTG as a graph",
                              fluid=True,
                              children=[
                                    p1_wgt_graphMTG    
                            ])


p1_col2 = v.Col(cols=12, sm=8, md=8,
                children=[
                          p1_panel_table_mtg,
                          v.Divider(),
                          p1_panel_graph_mtg,
                      ])


p1_content1 = v.Row(children=[
                              p1_col1,
                              p1_col2,
                          ])

# # ----------------------------------------------------------------
# # PAGE 1: Log
# # ----------------------------------------------------------------

p1_log_content = v.Textarea(background_color="black",
                            dark=True,
                            auto_grow=True,
                            value= "ICI s'afficheront les messages d'erreurs et les infos sur les imports"
                           )

p1_log = v.Row(children=[
                  p1_log_content
              ])

# # ----------------------------------------------------------------
# # PAGE 1: TAB 1
# # ----------------------------------------------------------------

p1_tab1 = v.Row(children=[
                    v.Col(col=12, sm=11, md=11,
                          children=[p1_content1,
#                                     p1_log,
                                   ])
                ])
# p1_tab1

In [8]:
# # ----------------------------------------------------------------
# # PAGE 1: Files description
# # ----------------------------------------------------------------

def print_files_description():
    with p1_wgt_files_description:
        p1_wgt_files_description.clear_output()
        # nb of files selected
        nb_files = len(p1_wgt_files_selection.v_model)

        # experiments names:
        exp_names = set(all_mtg.property('Experiment_name').values())

        # genotypes: 
        genotype_names = get_genotypes(all_mtg)

        # mtg.properties: + list of possible values
        properties = [(p, set(all_mtg.property(p).values())) for p in all_mtg.property_names() if p not in ['edge_type', 'index', 'label', 
                                                                                                       'Experiment_name', 'Genotype', 
                                                                                                       '_line', 'DBI']]
        print('Number of files selected:', nb_files)
        print('Experiment names:', exp_names)
        print('List of genotypes:', genotype_names)
        for genotype in genotype_names:
            print(genotype, ':', len(get_vid_of_genotype(all_mtg, [genotype])), ' plants')
        print('List of properties:', all_mtg.property_names())
        print('\n')
        print('List of properties values:')

        for p in properties:
            if p[1]:
                if isinstance(next(iter(p[1])), float):
                    print(p[0], ' min:', min(p[1]), 'max:', max(p[1]), 'mean:', np.mean(list(p[1])), 'std:', np.std(list(p[1])), 'Q1:', np.quantile(list(p[1]), 0.25),'Q2:', np.quantile(list(p[1]), 0.50),'Q3:', np.quantile(list(p[1]), 0.75),)
                else:
                    print(p[0], ': ', p[1])


p1_wgt_files_description = widgets.Output(layout=layout_output_wgt)

p1_panel_files_description = v.Container(
                              fluid=False,
                              children=[
                                    p1_wgt_files_description    
                            ])

# # ----------------------------------------------------------------
# # PAGE 1: TAB 2
# # ----------------------------------------------------------------

p1_tab2 = v.Row(children=[v.Col(cols=12, sm=3, md=3,
                                children=[p1_wgt_files_selection]),
                          v.Col(cols=12, sm=7, md=9,
                                children=[p1_panel_files_description])
                          ])


In [9]:
# # ----------------------------------------------------------------
# # PAGE 1: main container 
# # ----------------------------------------------------------------

p1 = v.Tabs( 
            children=[
            v.Tab(children=['Import Files']),
            v.Tab(children=['Files Description']),
            v.TabItem(children=[
                p1_tab1
            ]),
            v.TabItem(children=[
                p1_tab2
            ])
        ])

p1_container_main = v.Container(fluid=True, 
                                   class_='grid-list-md box',
                                   children=[
#                                        v.Banner(children=["Import MTG files"]),
                                       p1
                                   ])
# p1_container_main

# Visualization

In [10]:
# # ----------------------------------------------------------------
# # PAGE 2: Select Genotype 
# # ----------------------------------------------------------------

def save_p2():
    pdf = PDF()
    pdf.set_title('Visualization')
    pdf.add_page()
    
    pdf.output('p2.pdf', 'F')
    


p2_wgt_genotype_selection_3D = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Genotype",
            truncate_length=22)

p2_wgt_genotype_selection_2D = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Genotype",
            truncate_length=22)

p2_wgt_parameters = v.Select(items=[True, False], 
                        label="Group by color", 
                        multiple=False, chips=True, 
                        v_model=False)


p2_row1 = v.Row(children=[v.Col(children=[p2_wgt_genotype_selection_3D]),
                          v.Col(children=[p2_wgt_parameters])
                         ])

p2_row12 = v.Row(children=[v.Col(children=[p2_wgt_genotype_selection_2D]),                          
                         ])


In [11]:
# # ----------------------------------------------------------------
# # PAGE 2: 3D visualization 
# # ----------------------------------------------------------------

def plot_3D_growth_developement(mtg, vids=[]):
    if not vids:
        return None
    else:
        mtg.properties()['order'] = orders(mtg)
        scene=visu3d.plot3d(mtg,by=["Sample_date"],hide_leaves=False,display=False, vids=vids)
        p = PlantGL(scene, group_by_color=p2_wgt_parameters.v_model)
        return p 

def print_3D_growth_developement(mtg, genotype):
    if mtg:
        with p2_wgt_3D_growth_developement:
            p2_wgt_3D_growth_developement.clear_output()
            vids_selected = get_vid_of_genotype(all_mtg, genotypes=[genotype])
            print('3D growth developement')
            print(genotype, ': ')
            display(plot_3D_growth_developement(all_mtg, vids=vids_selected))
    else:
        with p2_wgt_3D_growth_developement:
            p2_wgt_3D_growth_developement.clear_output()
            print('Select a Genotype')

def plot_3D_floral_intensity(mtg, vids=[]):
    mtg.properties()['order'] = orders(mtg)
    scene=visu3d.plot3d(mtg,by=["Sample_date"],hide_leaves=True,display=False, vids=vids)
    p = PlantGL(scene, group_by_color=p2_wgt_parameters.v_model)
    return p 

def print_3D_floral_intensity(mtg, genotype):
    if mtg:
        with p2_wgt_3D_floral_intensity:
            p2_wgt_3D_floral_intensity.clear_output()
            vids_selected = get_vid_of_genotype(all_mtg, genotypes=[genotype])
            print('3D floral_intensity ')
            print(genotype, ': ')
            display(plot_3D_floral_intensity(all_mtg, vids=vids_selected))
            
    else:
        with p2_wgt_3D_growth_developement:
            p2_wgt_3D_growth_developement.clear_output()
            print('Select a Genotype')    
        

def on_change_3d(widget, event, data):
    # select the mtg from genotype
    print_3D_floral_intensity(all_mtg, data)
    print_3D_growth_developement(all_mtg, data)
    

p2_wgt_3D_growth_developement = widgets.Output(layout=layout_output_wgt)

#
# print_3D_growth_developement()

p2_panel_3D_growth_developement = v.Container(
                              style_='max-height: 120px;',
                              pa="0",
                              fluid=True,
                              children=[
                                    p2_wgt_3D_growth_developement    
                            ])

p2_col2 = v.Col(cols=12, sm=12, md=6, class_="pa-0",
                children=[
                          p2_panel_3D_growth_developement,
                      ])


p2_wgt_3D_floral_intensity = widgets.Output(layout=layout_output_wgt)

p2_panel_3D_floral_intensity = v.Container(
#                               style_='max-height: 1200px',
                              fluid=True,
                              children=[
                                    p2_wgt_3D_floral_intensity    
                            ])

p2_col3 = v.Col(cols=12, sm=12, md=6, class_="pa-0",
                children=[
                          p2_panel_3D_floral_intensity,
                      ])
p2_panel_3D = v.Row(children=[p2_col2, p2_col3])

p2_tab1 = v.Row(children=[v.Col(col=12, sm=11, md=11,
                                children=[p2_row1,
                                      p2_panel_3D
                                      ])
                         ])

p2_wgt_genotype_selection_3D.on_event("change", on_change_3d)

p2_wgt_parameters.on_event("change", on_change_3d)

# p2_tab1

In [12]:
# # ----------------------------------------------------------------
# # PAGE 2: 2D visualization 
# # ----------------------------------------------------------------

def plot_2D_single_p(mtg):
    return mtg
    
    
def print_2D_single_p():
    with p2_wgt_2D_single_p:
        p2_wgt_2D_single_p.clear_output()
        print('2D Single plant')
        print(plot_2D_single_p(0))
            
# p2_wgt_2D_single_p = widgets.Output(layout={'border': '1px solid black'})
p2_wgt_2D_single_p = v.Img(src="2D_single_p.png")

#

p2_panel_2D_single_p = v.Container(
                              fluid=True,
                              children=[
                                    p2_wgt_2D_single_p    
                            ])

p2_col4 = v.Col(cols=12, sm=12, md=6, class_="pa-0",
                children=[
                          p2_panel_2D_single_p,
                      ])

p2_wgt_2D_most_central = v.Img(src="2D_most_central.png")

#

p2_panel_2D_most_central = v.Container(
                              fluid=True,
                              children=[
                                    p2_wgt_2D_most_central    
                            ])

p2_col5 = v.Col(cols=12, sm=12, md=6, class_="pa-0",
                children=[
                          p2_panel_2D_most_central,
                      ])


p2_panel_2D = v.Row(children=[p2_col4, p2_col5])


p2_tab2 = v.Row(children=[v.Col(col=12, sm=11, md=11,
                                children=[p2_row12,
                                      p2_panel_2D
                                      ])
                         ])
# p2_tab2

In [13]:
# # ----------------------------------------------------------------
# # PAGE 2: main container 
# # ----------------------------------------------------------------

def on_change_tab_3d(widget, event, data):
    p2_wgt_genotype_selection_3D.v_model = ""
    
p2_t1 = v.Tab(children=['3D Visualization'])

p2_t1.on_event('change', on_change_tab_3d)

p2 = v.Tabs( 
            children=[
            v.Tab(children=['2D Visualization']),
            p2_t1,
            v.TabItem(children=[
                p2_tab2
            ]),
            v.TabItem(children=[
                p2_tab1
            ]), 
        ])

p2_container_main = v.Container(fluid=True, 
                                   class_='grid-list-md box',
                                   children=[
                                       p2
                                   ])
# p2_container_main

# Plant Scale

In [14]:
# # ----------------------------------------------------------------
# # PAGE 3: Select Genotype 
# # ----------------------------------------------------------------
        
p3_wgt_export = v.Btn(children=['Export table'])

p3_wgt_genotypes_selection_t1 = v.Select(items=[],
            chips=True, 
            multiple=True,
            counter=True,
            v_model="",
            label="Select Genotypes",
            truncate_length=22)

p3_col1 = v.Col(cols=12, sm=3, md=3,
                children=[
                          p3_wgt_genotypes_selection_t1,
                          p3_wgt_export
                      ])

# # ----------------------------------------------------------------
# # PAGE 3: Create dataframe 
# # ----------------------------------------------------------------

def on_change_genotype_p3_t1(widget, event, data):
    # update table
    df=pd.DataFrame()
    if all_mtg:
        vids=get_vid_of_genotype(all_mtg, genotypes=data)
        df = extract_at_plant_scale(all_mtg, vids=vids)
    p3_wgt_df_plantscale.df = df
    
    # update descriptors
    with p3_wgt_descriptors:
        p3_wgt_descriptors.clear_output()
        display(df.describe())
    
    
p3_wgt_df_plantscale = qgrid.show_grid(pd.DataFrame(), show_toolbar=False, 
                                  grid_options={'forceFitColumns': False, 'editable':True, 'defaultColumnWidth':50})


p3_panel_table = v.Container(
#                               style_='width: 600px; height: 300px',
                              fluid=True,
                              children=[
                                    p3_wgt_df_plantscale    
                            ])

p3_wgt_descriptors = widgets.Output(layout=layout_output_wgt)

p3_panel_descriptors =v.Container(fluid=True,
                              children=[
                                    p3_wgt_descriptors    
                                ])

p3_col2 = v.Col(cols=12, sm=7, md=9,
                children=[
                          p3_panel_table,
                          p3_panel_descriptors,
                      ])

p3_wgt_genotypes_selection_t1.on_event('change', on_change_genotype_p3_t1)

p3_tab1 = v.Row(children=[p3_col1,
                          p3_col2,
                          ])

In [15]:
# # ----------------------------------------------------------------
# # PAGE 3: Single Geno analyses 
# # ----------------------------------------------------------------

def transfert_figure_pie(source, cible):
    cible.data = []
    cible.layout = {}
    cible.update_layout(source.layout)
    for trace in source.data:
        cible.add_pie(**trace.to_plotly_json())

def on_change_genotype_p3_t2(widget, event, data):
    genotype=p3_wgt_genotypes_selection_t2.v_model
    vids=get_vid_of_genotype(all_mtg, [genotype])
    df=extract_at_plant_scale(all_mtg, vids=vids)
    param = list(df.columns)
    param.remove("Genotype")
    param.remove('date')
    p3_wgt_date_selection.items=list(df.date.unique())
    p3_wgt_date_selection.v_model=""
    p3_wgt_parameter_selection.items=param
    

def on_change_date_p3(widget, event, data):

    genotype=p3_wgt_genotypes_selection_t2.v_model
    vids=get_vid_of_genotype(all_mtg, [genotype])
    df=extract_at_module_scale(all_mtg, vids=vids)
    
    tmp=df2waffle(df, index='order', 
              date=data, 
              variable='stage', 
              crosstab=True)
    
    try:
        fig = plot_pie(tmp)
        fig.update_layout(title="Percentage representation of each stage at one date")
        transfert_figure_pie(fig,p3_pie)
    except ValueError:
        pass
            
            
    
def on_change_parameter_p3(widget, event, data):
#     if date ...
    genotype=p3_wgt_genotypes_selection_t2.v_model
    vids=get_vid_of_genotype(all_mtg, [genotype])
    df=extract_at_plant_scale(all_mtg, vids=vids)

    res = df.groupby('date').mean()
    
    p=res.iplot(kind = "line", 
              mode='lines+markers', 
              y=data,
              xTitle="Dates",
              yTitle=data,
#               yTitle="Probability",
#               title="Relative distribution of complete (True) and incomplete (False) module as function of date for {}".format(genotype),
              asFigure=True
             )
    transfert_figure(p, p3_plot) 


p3_wgt_genotypes_selection_t2 = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Genotype",
            truncate_length=22)

p3_wgt_date_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Date",
            truncate_length=22)

p3_wgt_parameter_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Parameter",
            truncate_length=22)



p3_select_genotype = v.Row(children=[p3_wgt_genotypes_selection_t2,
                                   p3_wgt_date_selection,
                                   p3_wgt_parameter_selection,
                                   ])

p3_plot = go.FigureWidget(layout=layout_gofigure)
p3_pie = go.FigureWidget(layout=layout_gofigure)

p3_tab2 = v.Row(children=[v.Col(
                            children=[
                                p3_select_genotype,
                                v.Row(children=[p3_plot, p3_pie]),
                              ]),
                        ])

p3_wgt_genotypes_selection_t2.on_event('change', on_change_genotype_p3_t2)
p3_wgt_date_selection.on_event('change', on_change_date_p3)
p3_wgt_parameter_selection.on_event('change', on_change_parameter_p3)


In [16]:
p3 = v.Tabs( 
            children=[
            v.Tab(children=['Export']),
            v.Tab(children=['Analyses']),
            v.TabItem(children=[
                p3_tab1,]),
            v.TabItem(children=[p3_tab2]),

        ])


p3_container_main = v.Container(fluid=True, 
                                   class_='grid-list-md box',
                                   children=[
#                                        v.Banner(children=["Import MTG files"]),
                                       p3
                                   ])


# Module Scale

In [17]:
def save_p4():
    pdf = PDF()
    pdf.set_title('Module scale')
    pdf.add_page()
    if all_mtg:
        # Single genotype
        pdf.add_btext("Occurence of successive module order")
        for genotype in p4_wgt_genotypes_selection_t1.items:
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_module_scale(all_mtg, vids=vids)
            fig = plot_module_occurence_module_order_along_time(df, genotype)
            tmp_path = "figs/"+genotype+"_occurence_order"+".png"
            fig.write_image(tmp_path)
            pdf.add_image(tmp_path)

        pdf.add_btext("Relative distribution of complete and incomplete module per order")
        for genotype in p4_wgt_genotypes_selection_t1.items:
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_module_scale(all_mtg, vids=vids)
            fig = plot_module_distribution_complete_incomplete_module_order(df, genotype)
            tmp_path = "figs/"+genotype+"_distribution_order"+".png"
            fig.write_image(tmp_path)
            pdf.add_image(tmp_path)

        pdf.add_btext("Relative distribution of complete and incomplete module per date")
        for genotype in p4_wgt_genotypes_selection_t1.items:
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_module_scale(all_mtg, vids=vids)
            fig = plot_module_distribution_complete_incomplete_date(df, genotype)
            tmp_path = "figs/"+genotype+"_distribution_date"+".png"
            fig.write_image(tmp_path)
            pdf.add_image(tmp_path)

        # multiple genotypes
        df=extract_at_module_scale(all_mtg)

        pdf.add_btext("Pointwise of nb of leaves")
        fig = plot_module_pointwisemean(df, "nb_total_leaves")
        tmp_path = "figs/nb_total_leaves"+".png"
        fig.write_image(tmp_path)
        pdf.add_image(tmp_path)

        pdf.add_btext("Pointwise of nb of flowers")
        fig = plot_module_pointwisemean(df, "nb_total_flowers")
        tmp_path = "figs/nb_total_flowers"+".png"
        fig.write_image(tmp_path)
        pdf.add_image(tmp_path)

        pdf.add_btext("Pointwise of nb of stolons")
        fig = plot_module_pointwisemean(df, "nb_stolons")
        tmp_path = "figs/nb_stolons"+".png"
        fig.write_image(tmp_path)
        pdf.add_image(tmp_path)

        pdf.add_btext("Branch crown")
        fig = plot_module_crown(df, "branch_crown")
        tmp_path = "figs/branch_crown"+".png"
        fig.write_image(tmp_path)
        pdf.add_image(tmp_path)

        pdf.add_btext("Extension crown")
        fig = plot_module_crown(df, "extension_crown")
        tmp_path = "figs/extension_crown"+".png"
        fig.write_image(tmp_path)
        pdf.add_image(tmp_path)
    
    else:
        pdf.add_text(" ")
    pdf.output('p4.pdf', 'F')
    

In [18]:
# # ----------------------------------------------------------------
# # PAGE 4: Select Genotype 
# # ----------------------------------------------------------------
        
p4_wgt_export = v.Btn(children=['Export table'])

p4_wgt_genotypes_selection_t1 = v.Select(items=[],
            chips=True, 
            multiple=True,
            counter=True,
            v_model="",
            label="Select Genotypes",
            truncate_length=22)

p4_col1 = v.Col(cols=12, sm=3, md=3,
                children=[
                          p4_wgt_genotypes_selection_t1,
                          p4_wgt_export
                      ])

In [19]:
# # ----------------------------------------------------------------
# # PAGE 4: Create dataframe 
# # ----------------------------------------------------------------

def on_change_genotype_p4(widget, event, data):
    # update table
    df=pd.DataFrame()
    if all_mtg:
        vids=get_vid_of_genotype(all_mtg, genotypes=data)
        df = extract_at_module_scale(all_mtg, vids=vids)
    p4_wgt_df_modulescale.df = df
    
    # update descriptors
    with p4_wgt_descriptors:
        p4_wgt_descriptors.clear_output()
        display(df.describe())
    
    
p4_wgt_df_modulescale = qgrid.show_grid(pd.DataFrame(), show_toolbar=False, 
                                  grid_options={'forceFitColumns': False, 'editable':True, 'defaultColumnWidth':50})


p4_panel_table = v.Container(
#                               style_='width: 600px; height: 300px',
                              fluid=True,
                              children=[
                                    p4_wgt_df_modulescale    
                            ])

p4_wgt_descriptors = widgets.Output(layout=layout_output_wgt)

p4_panel_descriptors =v.Container(fluid=True,
                              children=[
                                    p4_wgt_descriptors    
                                ])

p4_col2 = v.Col(cols=12, sm=7, md=9,
                children=[
                          p4_panel_table,
                          p4_panel_descriptors,
                      ])

p4_wgt_genotypes_selection_t1.on_event('change', on_change_genotype_p4)

In [20]:
p4_tab1 = v.Row(children=[p4_col1,
                          p4_col2,
                          ])
# p4_tab1

In [21]:
# # ----------------------------------------------------------------
# # PAGE 4: Analyses single genotype 
# # ----------------------------------------------------------------

def plot_module_occurence_module_order_along_time(df, genotype):
    geno_frequency = occurence_module_order_along_time(data= df,frequency_type= "cdf")
    p=geno_frequency.iplot(kind = "line", 
                          mode='lines+markers', 
                          xTitle="order",
                          yTitle="cdf",
                          title="Occurence of successive module order for {}".format(genotype),
                          asFigure=True
                         )
    return p 

def plot_module_distribution_complete_incomplete_module_order(df, genotype):
    res=pd.crosstab(index= df["order"], columns= df["complete_module"],normalize="index")
    res.columns = ["incomplete","complete"]
    
    p=res.iplot(kind = "line", 
                      mode='lines+markers', 
                      xTitle="Order",
                      yTitle="Probability",
                      title="Relative distrivution of complete (True) and incomplete (False) module as function of module order for {}".format(genotype),
                      asFigure=True
                     )
    return p 

def plot_module_distribution_complete_incomplete_date(df, genotype):
    res=pd.crosstab(index=df["date"], columns= df["complete_module"],normalize="index")
    res.columns = ["incomplete","complete"]    

    p=res.iplot(kind = "line", 
                  mode='lines+markers', 
                  xTitle="Dates",
                  yTitle="Probability",
                  title="Relative distribution of complete (True) and incomplete (False) module as function of date for {}".format(genotype),
                  asFigure=True
                 )
    return p 

def transfert_figure(source, cible):
    cible.data = []
    cible.layout = {}
    cible.update_layout(source.layout)
    for trace in source.data:
        cible.add_scatter(**trace.to_plotly_json())

def print_single_genotype_plots():
    genotype=p4_wgt_genotypes_selection_t2.v_model
    vids=get_vid_of_genotype(all_mtg, [genotype])
    df=extract_at_module_scale(all_mtg, vids=vids)
    if genotype:
        # plot occurence module order
        fig = plot_module_occurence_module_order_along_time(df, genotype)
        transfert_figure(fig, p4_wgt_occurence)
        # plot occurence module order
        fig = plot_module_distribution_complete_incomplete_module_order(df, genotype)
        transfert_figure(fig, p4_wgt_distribution_module_order)
        # plot occurence module order
        fig = plot_module_distribution_complete_incomplete_date(df, genotype)
        transfert_figure(fig, p4_wgt_distribution_date)
    else:
        with p4_wgt_occurence:
            p4_wgt_occurence.clear_output()
            print('Select a Genotype')
    

def on_change_single_genotype(widget, event, data):
    print_single_genotype_plots()
        

def create_download_link( df, title = "Download CSV file", filename = "selected_mtg.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


def on_click_export_411(widget, event, data):
    genotype=p4_wgt_genotypes_selection_t2.v_model
    if genotype:
        vids=get_vid_of_genotype(all_mtg, [genotype])
        df=extract_at_module_scale(all_mtg, vids=vids)
        geno_frequency = occurence_module_order_along_time(data= df,frequency_type= "cdf")
        with p4_link_export_t11:
            p4_link_export_t11.clear_output()
            display(create_download_link(geno_frequency))
        
        
def on_click_export_412(widget, event, data):
    genotype=p4_wgt_genotypes_selection_t2.v_model
    if genotype:
        vids=get_vid_of_genotype(all_mtg, [genotype])
        df=extract_at_module_scale(all_mtg, vids=vids)
        res=pd.crosstab(index= df["order"], columns= df["complete_module"],normalize="index")
        res.columns = ["incomplete","complete"]
        with p4_link_export_t12:
            p4_link_export_t12.clear_output()
            display(create_download_link(res))
        
def on_click_export_413(widget, event, data):
    genotype=p4_wgt_genotypes_selection_t2.v_model
    if genotype:
        vids=get_vid_of_genotype(all_mtg, [genotype])
        df=extract_at_module_scale(all_mtg, vids=vids)
        res=pd.crosstab(index=df["date"], columns= df["complete_module"],normalize="index")
        res.columns = ["incomplete","complete"]    
        with p4_link_export_t13:
            p4_link_export_t13.clear_output()
            display(create_download_link(res))
        
    
        
p4_wgt_genotypes_selection_t2 = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Genotype",
            truncate_length=22)

p4_wgt_occurence = go.FigureWidget(layout=layout_gofigure)
p4_wgt_distribution_module_order = go.FigureWidget(layout=layout_gofigure)
p4_wgt_distribution_date = go.FigureWidget(layout=layout_gofigure)

p4_link_export_t11 = widgets.Output(layout=layout_output_wgt)
p4_link_export_t12 = widgets.Output(layout=layout_output_wgt)
p4_link_export_t13 = widgets.Output(layout=layout_output_wgt)

p4_wgt_export_t11 = v.Btn(children=['Export table'])
p4_wgt_export_t12 = v.Btn(children=['Export table'])
p4_wgt_export_t13 = v.Btn(children=['Export table'])


p4_panel_single_genotype = v.Container(fluid=True,
                              children=[
                                    p4_wgt_occurence,
                                    v.Row(children=[p4_wgt_export_t11,p4_link_export_t11]),
                                    p4_wgt_distribution_module_order,
                                    v.Row(children=[p4_wgt_export_t12,p4_link_export_t12]),
                                    p4_wgt_distribution_date,
                                    v.Row(children=[p4_wgt_export_t13,p4_link_export_t13]),
                                ])


p4_col3 = v.Col(col=12, sm=11, md=11,
                children=[
                    p4_wgt_genotypes_selection_t2,
                    p4_panel_single_genotype
                  ])

p4_tab2 = v.Row(children=[p4_col3
                          ])


p4_wgt_genotypes_selection_t2.on_event("change", on_change_single_genotype)

p4_wgt_export_t11.on_event('click', on_click_export_411)
p4_wgt_export_t12.on_event('click', on_click_export_412)
p4_wgt_export_t13.on_event('click', on_click_export_413)

# p4_tab2

In [22]:
# # ----------------------------------------------------------------
# # PAGE 4: Analyses multiple genotype 
# # ----------------------------------------------------------------

def crowntype_distribution(data, crown_type,):
    """
    parameters:
    -----------
    data: panda dataframe issue from extraction of data at module scale
    variable: type of branch crown (extension_crown or branch_crown)

    return:
    -------
    a dataframe containing relative frequency values by genotype and order for extension and branch crown
    and a relative frequency distribution plot

    """
    df= pd.crosstab(index= [data.Genotype, data.order],
                    columns= data.type_of_crown,
                    normalize="index")   
    df.columns=["Main", "extension_crown", "branch_crown"]

    return df


def crowntype_plotly(data, crown_type, title, ylab,):
    fig=go.Figure()
    for genotype in list(data.unstack(level=0)[crown_type].columns):
        fig.add_trace(go.Scatter(
                x=list(data.unstack(level=0).index),
                y=list(data.unstack(level=0)[crown_type][genotype]),
                name=genotype,
                ))

    fig.update_layout(yaxis_title=ylab,
                      xaxis_title="Order",
                      title=title,
                      )
    return go.FigureWidget(fig)

def pointwisemean_plotly(data_mean, data_sd, variable,title,ylab,):
    fig=go.Figure()
    for genotype in list(data_mean.unstack(level=0)[variable].columns):
        sd_error = list(data_sd.unstack(level=0)[variable][genotype])
        fig.add_trace(go.Scatter(
                x=list(data_mean.unstack(level=0).index),
                y=data_mean.unstack(level=0)[variable][genotype],
                name=genotype,
                error_y=dict(type='data', 
                             array=sd_error, 
                             thickness=1,
                             visible=True)
                     ))

    fig.update_layout(yaxis_title=ylab,
                      xaxis_title="Order",
                      title=title,
                      )
    return go.FigureWidget(fig)



def plot_module_pointwisemean(df, var):
    Mean= df.groupby(["Genotype", "order"]).mean()
    sd= df.groupby(["Genotype", "order"]).std()
#     variable = "nb_total_{}".format(var)
    fig = pointwisemean_plotly(data_mean=Mean,
                   data_sd=sd,
                   variable=var,
                   title= "Pointwisemean of {}".format(var),
                   ylab="Mean {}".format(var),)

    return fig

def plot_module_crown(df, var):
    ctd = crowntype_distribution(data= df, crown_type=var)
    ctd = ctd[ctd.index.get_level_values('order')!=0]
    fig= crowntype_plotly(data=ctd,
                 crown_type=var, 
                 title="Relative frequency of {}".format(var),
                 ylab="Relative frequency")
    return fig


def print_multiple_genotypes_plots():
    df=extract_at_module_scale(all_mtg)
    if all_mtg:
        # plot pointwisemean leaves
        fig = plot_module_pointwisemean(df, "nb_total_leaves")
        transfert_figure(fig, p4_wgt_pointwisemean_leaves)
        # plot pointwisemean flowers
        fig = plot_module_pointwisemean(df, "nb_total_flowers")
        transfert_figure(fig, p4_wgt_pointwisemean_flowers)
        # plot pointwisemean stolons
        fig = plot_module_pointwisemean(df, "nb_stolons")
        transfert_figure(fig, p4_wgt_pointwisemean_stolons)
        # crowntype branch
        fig = plot_module_crown(df, "branch_crown")
        transfert_figure(fig, p4_wgt_branch_crown)
        # crowntype extension
        fig = plot_module_crown(df, "extension_crown")
        transfert_figure(fig, p4_wgt_extension_crown)


def on_click_export_421(widget, event, data):
    if all_mtg:
        df=extract_at_module_scale(all_mtg)
        Mean= df.groupby(["Genotype", "order"]).mean()
        with p4_link_export_t21:
            p4_link_export_t21.clear_output()
            display(create_download_link(Mean))

            
def on_click_export_422(widget, event, data):
    if all_mtg:
        df=extract_at_module_scale(all_mtg)
        Mean= df.groupby(["Genotype", "order"]).mean()
        with p4_link_export_t22:
            p4_link_export_t22.clear_output()
            display(create_download_link(Mean))
            
def on_click_export_423(widget, event, data):
    if all_mtg:
        df=extract_at_module_scale(all_mtg)
        Mean= df.groupby(["Genotype", "order"]).mean()
        with p4_link_export_t23:
            p4_link_export_t23.clear_output()
            display(create_download_link(Mean))
            
def on_click_export_424(widget, event, data):
    if all_mtg:
        df=extract_at_module_scale(all_mtg)
        ctd = crowntype_distribution(data= df, crown_type="branch_crown")
        ctd = ctd[ctd.index.get_level_values('order')!=0]
        with p4_link_export_t24:
            p4_link_export_t24.clear_output()
            display(create_download_link(ctd))
            
def on_click_export_425(widget, event, data):
    if all_mtg:
        df=extract_at_module_scale(all_mtg)
        ctd = crowntype_distribution(data= df, crown_type="extension_crown")
        ctd = ctd[ctd.index.get_level_values('order')!=0]
        with p4_link_export_t25:
            p4_link_export_t25.clear_output()
            display(create_download_link(ctd))
            

p4_wgt_pointwisemean_leaves = go.FigureWidget(layout=layout_gofigure)
p4_wgt_pointwisemean_flowers = go.FigureWidget(layout=layout_gofigure)
p4_wgt_pointwisemean_stolons = go.FigureWidget(layout=layout_gofigure)
p4_wgt_branch_crown = go.FigureWidget(layout=layout_gofigure)
p4_wgt_extension_crown = go.FigureWidget(layout=layout_gofigure)


p4_link_export_t21 = widgets.Output(layout=layout_output_wgt)
p4_link_export_t22 = widgets.Output(layout=layout_output_wgt)
p4_link_export_t23 = widgets.Output(layout=layout_output_wgt)
p4_link_export_t24 = widgets.Output(layout=layout_output_wgt)
p4_link_export_t25 = widgets.Output(layout=layout_output_wgt)

p4_wgt_export_t21 = v.Btn(children=['Export table'])
p4_wgt_export_t22 = v.Btn(children=['Export table'])
p4_wgt_export_t23 = v.Btn(children=['Export table'])
p4_wgt_export_t24 = v.Btn(children=['Export table'])
p4_wgt_export_t25 = v.Btn(children=['Export table'])


p4_wgt_export_t21.on_event('click', on_click_export_421)
p4_wgt_export_t22.on_event('click', on_click_export_422)
p4_wgt_export_t23.on_event('click', on_click_export_423)
p4_wgt_export_t24.on_event('click', on_click_export_424)
p4_wgt_export_t25.on_event('click', on_click_export_425)


p4_panel_multiple_genotypes = v.Container(fluid=True,
                              children=[
                                    p4_wgt_pointwisemean_leaves,
                                    v.Row(children=[p4_wgt_export_t21,p4_link_export_t21]),
                                    p4_wgt_pointwisemean_flowers,
                                    v.Row(children=[p4_wgt_export_t22,p4_link_export_t22]),
                                    p4_wgt_pointwisemean_stolons,
                                    v.Row(children=[p4_wgt_export_t23,p4_link_export_t23]),
                                    p4_wgt_branch_crown,
                                    v.Row(children=[p4_wgt_export_t24,p4_link_export_t24]),
                                    p4_wgt_extension_crown,
                                    v.Row(children=[p4_wgt_export_t25,p4_link_export_t25]),
                                ])

p4_col4 = v.Col(col=12, sm=11, md=11,
                children=[
#                     p1_wgt_genotypes_selection,
                    p4_panel_multiple_genotypes
                  ])

p4_tab3 = v.Row(children=[p4_col4
                          ])

# p4_tab3

In [23]:
def on_change_module_waffle_genotype(widget, event, data):
    genotype=p4_wgt_genotypes_selection_t4.v_model
    vids=get_vid_of_genotype(all_mtg, [genotype])
    df=extract_at_module_scale(all_mtg, vids=vids)
    param = list(df.columns)
    param.remove("Genotype")
    param.remove('date')
    p4_wgt_date_selection.items=list(df.date.unique())
    p5_wgt_date_selection.v_model=""
    p4_wgt_parameter_selection.items=param
    
def on_change_module_waffle_date(widget, event, data):
    if not p4_wgt_parameter_selection.v_model:
        pass
    else:
        with p4_waffle:
            p4_waffle.clear_output()
            genotype=p4_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_module_scale(all_mtg, vids=vids)

            if p4_wgt_aggfunc_selection.v_model == 'concat_string':
                tmp=df2waffle(df, index='order', 
                          date=data, 
                          variable=p4_wgt_parameter_selection.v_model, 
                          aggfunc=lambda x: ' '.join(x),
                          crosstab=False)
            else:
                tmp=df2waffle(df, index='order', 
                          date=data, 
                          variable=p4_wgt_parameter_selection.v_model, 
                          aggfunc=p4_wgt_aggfunc_selection.v_model,
                          crosstab=False)

            fig = plot_waffle(tmp,
#                               layout=layout,
                              plot_func=p4_wgt_plot_type.v_model)
            display(fig)


    
def on_change_module_waffle_parameter(widget, event, data):
    if not p4_wgt_date_selection.v_model:
        pass
    else:
        with p4_waffle:
            p4_waffle.clear_output()
            genotype=p4_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_module_scale(all_mtg, vids=vids)

            if p4_wgt_aggfunc_selection.v_model == 'concat_string':
                tmp=df2waffle(df, index='order', 
                          date=p4_wgt_date_selection.v_model, 
                          variable=data, 
                          aggfunc=lambda x: ' '.join(x),
                          crosstab=False)
            else:
                tmp=df2waffle(df, index='order', 
                          date=p4_wgt_date_selection.v_model, 
                          variable=data, 
                          aggfunc=p4_wgt_aggfunc_selection.v_model,
                          crosstab=False)


            fig = plot_waffle(tmp,
#                               layout=layout,
                              plot_func=p4_wgt_plot_type.v_model)
            display(fig)

            
def on_change_module_aggfunc(widget, event, data):
    if p4_wgt_date_selection.v_model and p4_wgt_parameter_selection.v_model:
        with p4_waffle:
            p4_waffle.clear_output()
            genotype=p4_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_module_scale(all_mtg, vids=vids)

            
            if p4_wgt_aggfunc_selection.v_model == 'concat_string':
                tmp=df2waffle(df, index='order', 
                          date=p4_wgt_date_selection.v_model, 
                          variable=p4_wgt_parameter_selection.v_model, 
                          aggfunc=lambda x: ' '.join(x),
                          crosstab=False)
            else:
                tmp=df2waffle(df, index='order', 
                          date=p4_wgt_date_selection.v_model, 
                          variable=p4_wgt_parameter_selection.v_model, 
                          aggfunc=data,
                          crosstab=False)

            fig = plot_waffle(tmp,
#                               layout=layout,
                              plot_func=p4_wgt_plot_type.v_model)
            display(fig)

            
# def on_change_module_crosstab(widget, event, data):
#     if p4_wgt_date_selection.v_model and p4_wgt_parameter_selection.v_model:
#         with p4_waffle:
#             p4_waffle.clear_output()
#             genotype=p4_wgt_genotypes_selection_t4.v_model
#             vids=get_vid_of_genotype(all_mtg, [genotype])
#             df=extract_at_module_scale(all_mtg, vids=vids)

#             tmp=df2waffle(df, index='order', 
#                           date=p4_wgt_date_selection.v_model, 
#                           variable=p4_wgt_parameter_selection.v_model, 
#                           aggfunc=p4_wgt_aggfunc_selection.v_model,
#                           crosstab=data)
            
#             yticks_l =list(range(0,len(tmp.index)))
#             yticks_l.reverse()

#             layout={
#                 'xlabel': 'Plant',
#                 'xticks': range(0,len(tmp.columns)),
#                 'xticks_label': tmp.columns,
#                 'ylabel': 'Module',
#                 'yticks': range(0,len(tmp.index)),
#                 'yticks_label': yticks_l,
#             }



#                 fig = plot_waffle(tmp,
#                                   layout=layout,
#                                   plot_func=p4_wgt_plot_type.v_model)
#             display(fig)

            
            
def on_change_module_plot_type(widget, event, data):
    if p4_wgt_date_selection.v_model and p4_wgt_parameter_selection.v_model:
        with p4_waffle:
            p4_waffle.clear_output()
            genotype=p4_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_module_scale(all_mtg, vids=vids)

            if p4_wgt_aggfunc_selection.v_model == 'concat_string':
                tmp=df2waffle(df, index='order', 
                          date=p4_wgt_date_selection.v_model, 
                          variable=p4_wgt_parameter_selection.v_model, 
                          aggfunc=lambda x: ' '.join(x),
                          crosstab=False
                         )
            else:
                tmp=df2waffle(df, index='order', 
                          date=p4_wgt_date_selection.v_model, 
                          variable=p4_wgt_parameter_selection.v_model, 
                          aggfunc=p4_wgt_aggfunc_selection.v_model,
                          crosstab=False
                         )

            fig = plot_waffle(tmp,
#                               layout=layout,
                              plot_func=data)

            display(fig)


p4_wgt_genotypes_selection_t4 = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Genotype",
            truncate_length=22)

p4_wgt_date_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Date",
            truncate_length=22)

p4_wgt_parameter_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Parameter",
            truncate_length=22)

def concat_string():
    return ' '.join(x)
    
# LINK text & variable
p4_wgt_aggfunc_selection = v.Select(items=[{'text': 'Mean', 'value': 'mean'}, 
                                           {'text': 'Median', 'value': 'median'}, 
                                           {'text': 'Concatenate string', 'value': 'concat_string'}],
            chips=True, 
            multiple=False,
            v_model='mean',
            label="Select a function",
            truncate_length=22)

# TRUE FALSE
# p4_wgt_crosstab = v.Select(items=[True, False],
#             chips=True, 
#             multiple=False,
#             v_model=False,
#             label="Crosstab",
#             truncate_length=22)

p4_wgt_plot_type = v.Select(items=["matplotlib", 'plotly.imshow', 'plotly.heatmap'],
            chips=True, 
            multiple=False,
            v_model="matplotlib",
            label="Select a plot library",
            truncate_length=22)


p4_select_waffle = v.Row(children=[p4_wgt_genotypes_selection_t4,
                                   p4_wgt_date_selection,
                                   p4_wgt_parameter_selection,
                                   p4_wgt_aggfunc_selection,
#                                    p4_wgt_crosstab,
                                   p4_wgt_plot_type])

p4_waffle = widgets.Output(layout=layout_output_wgt)


p4_tab4 = v.Row(children=[v.Col(
                            children=[
                                p4_select_waffle,
                                p4_waffle,
                              ]),
                        ])

p4_wgt_genotypes_selection_t4.on_event('change', on_change_module_waffle_genotype)
p4_wgt_date_selection.on_event('change', on_change_module_waffle_date)
p4_wgt_parameter_selection.on_event('change', on_change_module_waffle_parameter)
p4_wgt_aggfunc_selection.on_event('change', on_change_module_aggfunc)
# p4_wgt_crosstab.on_event('change', on_change_module_crosstab)
p4_wgt_plot_type.on_event('change', on_change_module_plot_type)

In [24]:
def on_click_changetab(widget, event, data):
    print_multiple_genotypes_plots()

In [25]:
p4 = v.Tabs( 
            children=[
            v.Tab(children=['Data extraction']),
            v.Tab(children=['Single genotype']),
            v.Tab(children=['Multiple genotypes']),
            v.Tab(children=['Waffle']),
            v.TabItem(children=[
                p4_tab1
            ]),
            v.TabItem(children=[
                p4_tab2
            ]),
            v.TabItem(children=[
                p4_tab3
            ]),
            v.TabItem(children=[
                p4_tab4
            ]),  
        ])

p4.on_event("change", on_click_changetab)


p4_container_main = v.Container(fluid=True, 
                                   class_='grid-list-md box',
                                   children=[
#                                        v.Banner(children=["Import MTG files"]),
                                       p4
                                   ])

# p4_container_main

# Node Scale

In [26]:
def on_change_node_waffle_genotype(widget, event, data):
    genotype=p5_wgt_genotypes_selection_t4.v_model
    vids=get_vid_of_genotype(all_mtg, [genotype])
    df=extract_at_node_scale(all_mtg, vids=vids)
    param = list(df.columns)
    param.remove("Genotype")
    param.remove('date')
    p5_wgt_date_selection.items=list(df.date.unique())
    p5_wgt_date_selection.v_model=""
    p5_wgt_parameter_selection.items=param
    order = list(df.order.unique())
    order.append({'text':'All', 'value': None})
    p5_wgt_order_selection.items=order
    
def on_change_node_waffle_date(widget, event, data):
    if not p5_wgt_parameter_selection.v_model:
        pass
    else:
        with p5_waffle:
            p5_waffle.clear_output()
            genotype=p5_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_node_scale(all_mtg, vids=vids)
            tmp=df2waffle(df, index='rank', date=data, variable=p5_wgt_parameter_selection.v_model, order=p5_wgt_order_selection.v_model)

            # TODO: remove the legend here (which only apply to branching_type)
            l_names = {"1":"Stolon", 
               "2":"Vegetative bud",
               "3":"Initiated bud",
               "4":"Aborted bud",
               "5":"Floral bud",
               "6":"Branch crown"}

            fig=plot_waffle(tmp, 
#                             layout=layout,
                            legend_name=l_names,
                            plot_func=p5_wgt_plot_type.v_model)
            
            display(fig)
            
    
def on_change_node_waffle_parameter(widget, event, data):
    if not p5_wgt_date_selection.v_model:
        pass
    else:
        with p5_waffle:
            p5_waffle.clear_output()
            genotype=p5_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_node_scale(all_mtg, vids=vids)
            tmp=df2waffle(df, index='rank', date=p5_wgt_date_selection.v_model, variable=data, order=p5_wgt_order_selection.v_model)
            
            # TODO: remove the legend here (which only apply to branching_type)
            l_names = {"1":"Stolon", 
               "2":"Vegetative bud",
               "3":"Initiated bud",
               "4":"Aborted bud",
               "5":"Floral bud",
               "6":"Branch crown"}

            fig=plot_waffle(tmp, 
#                             layout=layout,
                            legend_name=l_names,
                            plot_func=p5_wgt_plot_type.v_model)
            display(fig)

def on_change_node_plot_type(widget, event, data):
    if not p5_wgt_date_selection.v_model:
        pass
    elif not p5_wgt_parameter_selection.v_model:
        pass
    else:
        with p5_waffle:
            p5_waffle.clear_output()
            genotype=p5_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_node_scale(all_mtg, vids=vids)
            tmp=df2waffle(df, index='rank', date=p5_wgt_date_selection.v_model, variable=p5_wgt_parameter_selection.v_model, order=p5_wgt_order_selection.v_model)
            
            # TODO: remove the legend here (which only apply to branching_type)
            l_names = {"1":"Stolon", 
               "2":"Vegetative bud",
               "3":"Initiated bud",
               "4":"Aborted bud",
               "5":"Floral bud",
               "6":"Branch crown"}

            fig=plot_waffle(tmp, 
#                             layout=layout,
                            legend_name=l_names,
                            plot_func=data)
            display(fig)
            
            
def on_change_node_order(widget, event, data):
    if not p5_wgt_date_selection.v_model or not p5_wgt_parameter_selection.v_model:
        pass
    else:
        with p5_waffle:
            p5_waffle.clear_output()
            genotype=p5_wgt_genotypes_selection_t4.v_model
            vids=get_vid_of_genotype(all_mtg, [genotype])
            df=extract_at_node_scale(all_mtg, vids=vids)
            tmp=df2waffle(df, index='rank', date=p5_wgt_date_selection.v_model, variable=p5_wgt_parameter_selection.v_model, order=data)
            
            # TODO: remove the legend here (which only apply to branching_type)
            l_names = {"1":"Stolon", 
               "2":"Vegetative bud",
               "3":"Initiated bud",
               "4":"Aborted bud",
               "5":"Floral bud",
               "6":"Branch crown"}

            fig=plot_waffle(tmp, 
#                             layout=layout,
                            legend_name=l_names,
                            plot_func=p5_wgt_plot_type.v_model)
            display(fig)
            
        
    
p5_wgt_genotypes_selection_t4 = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Genotype",
            truncate_length=22)

p5_wgt_date_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Date",
            truncate_length=22)

p5_wgt_parameter_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model="",
            label="Select Parameter",
            truncate_length=22)

p5_wgt_plot_type = v.Select(items=["matplotlib", 'plotly.imshow', 'plotly.heatmap'],
            chips=True, 
            multiple=False,
            v_model="matplotlib",
            label="Select a plot library",
            truncate_length=22)

p5_wgt_order_selection = v.Select(items=[],
            chips=True, 
            multiple=False,
            v_model=None,
            label="Select Order",
            truncate_length=22)

p5_select_waffle = v.Row(children=[p5_wgt_genotypes_selection_t4,
                                   p5_wgt_date_selection,
                                   p5_wgt_parameter_selection,
                                   p5_wgt_order_selection,
                                   p5_wgt_plot_type])

p5_waffle = widgets.Output(layout=layout_output_wgt)

p5_tab4 = v.Row(children=[v.Col(
                            children=[
                                p5_select_waffle,
                                p5_waffle,
                              ]),
                        ])

p5_wgt_genotypes_selection_t4.on_event('change', on_change_node_waffle_genotype)
p5_wgt_date_selection.on_event('change', on_change_node_waffle_date)
p5_wgt_parameter_selection.on_event('change', on_change_node_waffle_parameter)
p5_wgt_plot_type.on_event('change', on_change_node_plot_type)
p5_wgt_order_selection.on_event('change', on_change_node_order)

In [27]:
p5 = v.Tabs( 
            children=[
            v.Tab(children=['Waffle']),
            v.TabItem(children=[
                p5_tab4
            ]),
        ])


p5_container_main = v.Container(fluid=True, 
                                   class_='grid-list-md box',
                                   children=[
#                                        v.Banner(children=["Import MTG files"]),
                                       p5
                                   ])


# Save content

In [28]:
def save_all_data():
    save_p1()
    save_p2()
    save_p4()
    
def on_click_export_dashboard(widget, event, data):
    save_all_data()
    
p1_wgt_export.on_event('click', on_click_export_dashboard)

# Main container

In [29]:
container_main = v.Container(
    _metadata={'mount_id': 'content-main'}, 
    children=[
        p1_container_main
    ]
)
# container_main

# Navigation

In [30]:
# Navigation (menubar)
components_map = {
    'Import MTG': p1_container_main,
    'Visualization': p2_container_main,
    'Plant Scale': p3_container_main,
    'Module Scale': p4_container_main,
    'Node Scale': p5_container_main,
}

py_items = [
        { 'title': 'Import MTG', 'icon': 'mdi-view-dashboard'},
        { 'title': 'Visualization', 'icon': 'mdi-image'},
        { 'title': 'Plant Scale', 'icon': 'mdi-help-box'},
        { 'title': 'Module Scale', 'icon': 'mdi-view-dashboard'},
        { 'title': 'Node Scale', 'icon': 'mdi-image'},
      ]
      
class MyNavigation(v.VuetifyTemplate):
    items = List(py_items).tag(sync=True)
    
    template = Unicode('''  
        <v-list
          dense
          nav
        >
          <v-list-item
            v-for="item in items"
            :key="item.title"
            link
            @click="item_click(item)"
          >
            <v-list-item-icon>
              <v-icon>{{ item.icon }}</v-icon>
            </v-list-item-icon>
  
            <v-list-item-content>
              <v-list-item-title>{{ item.title }}</v-list-item-title>
            </v-list-item-content>
          </v-list-item>
        </v-list>
    ''').tag(sync=True)
    
    def vue_item_click(self, data):
        component = components_map[data['title']]
        container_main.children = [component]

navigation = MyNavigation()

container_nav = v.Container(
    _metadata={'mount_id': 'content-nav'}, 
    children=[
        navigation
    ]
)
container_nav

Container(children=[MyNavigation(events=['item_click'], items=[{'title': 'Import MTG', 'icon': 'mdi-view-dashb…

In [31]:
container_main

Container(children=[Container(children=[Tabs(children=[Tab(children=['Import Files']), Tab(children=['Files De…